In [1]:
# Author: Christopher Dare
# Date: 29th October 2020

# Evaluation of Rhymes produced by YouShen poetry model

## Plan of Attack
We need to break each word down into phonemes (graphemes).
Then, we will evaluate how similar the 2 words are by rhyme. (This is quite complex).

## Rhyme evaluation
For any 2 lines of a poem, we consider the last word in each line to score their rhyme similarity.
We can classify two lines as rhyme vs not rhyme, as well as quantify how much they rhyme with each other. (more on this needs to be thought through).

Consider the following poem line pairs:  

*The way he cleaned the bowl was thorough*  
*And then he came up to ask for more*

*In the morning Cinderella would fetch water,  
At night she was the dazzling queen of the gala*

*Everytime the sun rises to shine  
I am reminded of you, beautiful love of mine*

*My brother never trusted santa claus  
So his name always fostered banter wars*

From the pairs above, we can see that there is variable length of phonemes that consitute a valid rhyme. These can range from one phoneme in for a single word to many phonemes in one word...and even many phonemes in phrases.

Key question: What's the best mathematical way of determining that both words rhyme?


## Identifying phonemes
Phonemes can be identified from pronunciation dictionary mappings or prediction models.
Two promising packages with rich datasets that could prove useful for this projects are
1. CMU pronouncing dictionary (potentially faster since it's just a dictionary)
2. Big Phoney (Has preprocessing capabilities. Could be slower since it relies on a predictive model for words not existent in a dicitonary)
Other packages can be found here [on github](https://github.com/topics/phonetics)

To get quickly off the ground, we will use BigPhoney

In [13]:
!git clone https://github.com/repp/big-phoney.git big_phoney -q 
!cd big_phoney && python setup.py install

running install
running bdist_egg
running egg_info
creating big_phoney.egg-info
writing big_phoney.egg-info/PKG-INFO
writing dependency_links to big_phoney.egg-info/dependency_links.txt
writing requirements to big_phoney.egg-info/requires.txt
writing top-level names to big_phoney.egg-info/top_level.txt
writing manifest file 'big_phoney.egg-info/SOURCES.txt'
reading manifest file 'big_phoney.egg-info/SOURCES.txt'
writing manifest file 'big_phoney.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/big_phoney
copying big_phoney/phonetic_dictionary.py -> build/lib/big_phoney
copying big_phoney/prediction_model_utils.py -> build/lib/big_phoney
copying big_phoney/shared_constants.py -> build/lib/big_phoney
copying big_phoney/prediction_model.py -> build/lib/big_phoney
copying big_phoney/preprocessors.py -> build/lib/big_phoney
copying big_phoney/big_phoney.py -> build/lib/big_

Best match: wrapt 1.12.1
Processing wrapt-1.12.1.tar.gz
Writing /tmp/easy_install-gc0z0noo/wrapt-1.12.1/setup.cfg
Running wrapt-1.12.1/setup.py -q bdist_egg --dist-dir /tmp/easy_install-gc0z0noo/wrapt-1.12.1/egg-dist-tmp-8bx5layk
zip_safe flag not set; analyzing archive contents...
wrapt.__pycache__._wrappers.cpython-37: module references __file__
creating /home/dex/anaconda3/envs/aikit/lib/python3.7/site-packages/wrapt-1.12.1-py3.7-linux-x86_64.egg
Extracting wrapt-1.12.1-py3.7-linux-x86_64.egg to /home/dex/anaconda3/envs/aikit/lib/python3.7/site-packages
Adding wrapt 1.12.1 to easy-install.pth file

Installed /home/dex/anaconda3/envs/aikit/lib/python3.7/site-packages/wrapt-1.12.1-py3.7-linux-x86_64.egg
Searching for termcolor>=1.1.0
Reading https://pypi.org/simple/termcolor/
Best match: termcolor 1.1.0
Processing termcolor-1.1.0.tar.gz
Writing /tmp/easy_install-qpmmf_xc/termcolor-1.1.0/setup.cfg
Running termcolor-1.1.0/setup.py -q bdist_egg --dist-dir /tmp/easy_install-qpmmf_xc/termc

In [17]:
!pip install keras

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
  Using cached scipy-1.5.3-cp37-cp37m-manylinux1_x86_64.whl (25.9 MB)
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44448 sha256=694852ecec09c1d4b6ab7e18abaf664b804013a973f1945cc6d0d03f1f8432cf
  Stored in directory: /home/dex/.cache/pip/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653
Successfully built pyyaml


In [20]:
import keras
from big_phoney.big_phoney import BigPhoney
phoney = BigPhoney()
## Looks like the packages has dependency management issues. I'll come back to the code later.

ModuleNotFoundError: No module named 'keras'